In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Packages

In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 11000)
import matplotlib.pyplot as plt
import seaborn as sns

# Loading Data

In [ ]:
brand_data_org = pd.read_csv('/kaggle/input/used-car-dataset-ford-and-mercedes/ford.csv')

## Copying Datasets

In [ ]:
# Using a copy to play on the safe side, making sure the inital Dataframe won't be modified

In [ ]:
brand_data = brand_data_org.copy()

# Exploring Dataset

## Brand

In [ ]:
brand_data.info()

In [ ]:
# no nulls with appropriate data types

In [ ]:
brand_data.describe(include='all')

In [ ]:
# We have 23 unique models but most of the cars are Fiesta
# 3 different transmission types but most of the cars are Manual transmission
# 5 different fuel types but most of the cars are running on Petrol
# It is strange to see min mileage is 1 for used cars!!
# mpg max = 201.8!! It is not logic to have cars with more than 100 miles per gallon
# It is also strange to see engine size equal to zero but may be this can be true for electric cars only
# Let us explore the data in more details

## Data Exploration

### Data Fields Distribution Graphs

In [ ]:
fig, axes = plt.subplots(figsize=(15,8),nrows=2, ncols=3)
sns.distplot(brand_data['year'],ax=axes[0,0])
sns.distplot(brand_data['mileage'],ax=axes[0,1])
sns.distplot(brand_data['tax'],ax=axes[0,2])
sns.distplot(brand_data['mpg'],ax=axes[1,0])
sns.distplot(brand_data['engineSize'],ax=axes[1,1])
sns.distplot(brand_data['price'],ax=axes[1,2])

#### Observations

In [ ]:
## year:       Right Skewed, most used cars are between 2015 to 2020. We have cars after 2021. Will be dropped.

## mileage:    Right Skewed, since most cars are 5 years old. High correlation between mileage and year is clear.
##             it is also not logic to have mileage of used cars with values very close to zero or even below 50 miles 
##             (as an assumption)

## tax:        Nothing specific can be concluded, however no correlation with any other field can be noticed

## mpg:        Right Skewed, as mentioned above it is strange to have cars with such extremely excessive fuel consumption 
##             (>100 mpg!!) normally, 80 mpg are the max acceptable consumption which can be achieved by Hybrid cars. 
##             Accordingly, anything above 80 will be considered as outliers

## engineSize: Most used cars engine sizes at 1 

## price:      Right Skewed, since most cars are 5 years old. 

## Finding Correlations

In [ ]:
fields_correlation = sns.heatmap(brand_data.corr(), cmap="YlGnBu", annot=True)

### Correlogram for additional visulatization of the relation between each two features 

In [ ]:
sns.pairplot(brand_data, kind="reg",plot_kws={'line_kws':{'color':'red'}, 'scatter_kws': {'alpha': 0.1}},diag_kind="hist")

In [ ]:
## -Year & Price are +vely correlated - Recent cars sold at higher prices. Logic 
##  (refer to below graphs year vs mean mileage & year vs mean price)

## -Year & mileage are -vely correlated - As the cars gets older (Year decrease) the mileage driven by the car increases. Logic 
##  (refer to below graph mean price vs [mileage grouped by year])
##  As year and mileage are highly correlated (High Multicollinearity Excpeted), accordingly year will be dropped after 
##  finishing the preprocessing of data

## -Tax has no significant correlation with any of the other features except with mpg. Tax feature is likely to be dropped out.

## -mpg has no significant correlation with any othe features. 
##  It makes sense that cars with bigger engine size has lower mpg. Surpisingly, low correlation with engine size.  
##  This maybe due to the different fuel and transmission types.  

## -mpg feature is likely to be dropped as well. 

## -Year, mileage, enginesize are the most relevant features affecting car price.

### mileage vs mean price [grouped by year]

In [ ]:
brand_data=brand_data[brand_data['year']<2022]

In [ ]:
year_mile_avgprice=pd.DataFrame
year_mile_avgprice=year_mile_avgprice({'mean_price' : brand_data.groupby( ["year",'mileage'] ).mean()["price"]}).reset_index().sort_values(by='year',ascending=True)

In [ ]:
import itertools
marker = itertools.cycle((',', '+', '.', 'o', '*','s','p','X','d','h','v','^','<','>','1','2','3','4','8','|','_','x')) 
year_set=sorted(set(year_mile_avgprice['year']), reverse=True)
plt.figure(figsize=(17,8))
for year in year_set:
     selected_data = year_mile_avgprice.loc[year_mile_avgprice['year'] == year]
     plt.scatter(selected_data['mileage'], selected_data['mean_price'],marker = next(marker), label=year)
plt.xlabel('mileage')
plt.ylabel('mean price')
plt.legend()

In [ ]:
# Generally, as the the car gets older it is most likely to have been driven more mileages 
# and consequently th mean price get lower
# Despite the heatmap above shows that years is a bit higher in relation with price than mileage but 
# it is most likely to drop year column instead, because same year includes wide range of mileages
# however a specific mileage values covers a narrow range of years

## Further Exploration of Data (Categorical Features and Outliers Visulaization)

### Price

In [ ]:
brand_data.boxplot(column=['price'],figsize=(15,5),patch_artist = True,notch ='True',vert=0)

In [ ]:
## As shown above a large portion of the prices are considered as outliers
## This could be due to several reasons:
## 1. Wide range of models
## 2. Wide range of engine sizes
## 3. Wide range of years and mileage

## Therefore, it is not wise to drop all these outliers at this stage

In [ ]:
## Lets try to filter the data for prices above 25,000 and apply box plot again.

In [ ]:
brand_data_above45=brand_data[brand_data['price']>25000]
brand_data_above45.boxplot(column=['price'],figsize=(15,5),patch_artist = True,notch =False,vert=0)

In [ ]:
len(brand_data_above45['price'])

In [ ]:
# now we have only 527 outliers (above 25,000). Dataset total len=17965

In [ ]:
## Lets try to filter the data for prices above 40,000 and apply box plot again.

In [ ]:
brand_data_above75=brand_data[brand_data['price']>40000]
brand_data_above75.boxplot(column=['price'],figsize=(15,5),patch_artist = True,notch =False,vert=0)

In [ ]:
len(brand_data_above75['price'])

In [ ]:
# now we have only 13 outliers (above 40,000). Dataset total len=17965

In [ ]:
## Since the number of outliers are very small compared to the total length of the main dataframe (17965)
## It is also expected that this number may also reduce further after completing the preprocessing of data
## The remaining outliers will be eliminated as it will not have much impact on the model.

### Fuel Type and Mileage

In [ ]:
brand_data.boxplot(column=['mileage'], by=['fuelType'],figsize=(15,5),vert=0)

### Fuel Type and mpg

In [ ]:
brand_data.boxplot(column=['mpg'], by=['fuelType'],figsize=(15,5),patch_artist = True,notch =False,vert=0)

### mpg and transmission

In [ ]:
brand_data.boxplot(column=['mpg'], by=['transmission'],figsize=(15,5),patch_artist = True,notch ='True',vert=0)

In [ ]:
## it is very obvious that manual transmission provides more efficient performance
## No great difference between automatic and semi automatic mpg profiles however semi auto is slightly better than automatic

### mpg and engine size

In [ ]:
brand_data.boxplot(column=['mpg'], by=['engineSize'],figsize=(15,5))

### Count of Features

In [ ]:
fig, axes = plt.subplots(figsize=(20,12),nrows=2, ncols=2)
vis_models = brand_data.groupby(['model']).count()['price'].sort_values(ascending=False)
vis_ftyp = brand_data.groupby(['fuelType']).count()['price'].sort_values(ascending=False)
vis_trans = brand_data.groupby(['transmission']).count()['price'].sort_values(ascending=False)
vis_engS = brand_data.groupby(['engineSize']).count()['price']
vis_models.plot(ax=axes[0,0],kind='bar',title = 'Count')
vis_ftyp.plot(ax=axes[0,1],kind='bar',title = 'Count')
vis_engS.plot(ax=axes[1,0],kind='bar',title = 'Count')
vis_trans.plot(ax=axes[1,1],kind='bar',rot=360,title = 'Count')

### mpg, fuel type and transmission

In [ ]:
df=pd.DataFrame
df=brand_data[brand_data['transmission']=='Automatic']
df=df[df['fuelType']!='Hybrid']
df=df[df['fuelType']!='Electric']
df=df[df['fuelType']!='Other']
df=df[df['engineSize']>0]
plt.figure(figsize=(15,5))
plot01 = sns.boxplot(x='engineSize', y='mpg', hue='fuelType', notch=True, data=df, palette='bright')

In [ ]:
df=pd.DataFrame
df=brand_data[brand_data['transmission']=='Semi-Auto']
df=df[df['fuelType']!='Hybrid']
df=df[df['fuelType']!='Electric']
df=df[df['fuelType']!='Other']
df=df[df['engineSize']>0]
plt.figure(figsize=(15,5))
plot01 = sns.boxplot(x='engineSize', y='mpg', hue='fuelType', notch=True, data=df, palette='bright')

In [ ]:
df=pd.DataFrame
df=brand_data[brand_data['transmission']=='Manual']
df=df[df['fuelType']!='Hybrid']
df=df[df['fuelType']!='Electric']
df=df[df['fuelType']!='Other']
df=df[df['engineSize']>0]
plt.figure(figsize=(15,5))
plot01 = sns.boxplot(x='engineSize', y='mpg', hue='fuelType', notch=True, data=df, palette='bright')

### price and year 

In [ ]:
brand_data.boxplot(column=['price'], by=['year'],figsize=(15,5))

In [ ]:
## As shown the range of prices is higher for recent cars

### Price and model

In [ ]:
brand_data.boxplot(column=['price'], by=['model'],figsize=(15,5),rot=45)

In [ ]:
## it is clear that R8 is the most expensive car in audi, 
## it could have a great share of the overall price outliers indicated above along with other models

### Mean Price vs Other Features

In [ ]:
fig, axes = plt.subplots(figsize=(15,12),nrows=2, ncols=2)
vis_models = brand_data.groupby(['model']).mean()['price'].sort_values(ascending=False)
vis_trans = brand_data.groupby(['transmission']).mean()['price'].sort_values(ascending=False)
vis_ftyp = brand_data.groupby(['fuelType']).mean()['price'].sort_values(ascending=False)
vis_engS = brand_data.groupby(['engineSize']).mean()['price'].sort_values(ascending=False)
vis_models.plot(ax=axes[0,0],kind='bar',title = 'mean price')
vis_engS.plot(ax=axes[0,1],kind='bar',title = 'mean price')
vis_trans.plot(ax=axes[1,0],kind='bar',rot=360,title = 'mean price')
vis_ftyp.plot(ax=axes[1,1],kind='bar',rot=360,title = 'mean price')

## Preprocessing

### Tax feature

In [ ]:
# As concluded above that tax and mpg has no correlation with price or any other feature
# Tax and mpg columns will be dropped

In [ ]:
brand_notax=brand_data.drop(['tax'], axis=1)

In [ ]:
brand_notaxmpg=brand_notax.drop(['mpg'], axis=1)

In [ ]:
brand_notaxmpg.head()

### Mileage feature

In [ ]:
## there are some zero entries in the mileage. We are concerned with used cars
## Accordingly we will delete any car below 50 mileage records

In [ ]:
mile_min = 50
brand_mile = brand_notaxmpg[brand_notaxmpg['mileage']>=mile_min]

In [ ]:
sns.distplot(brand_mile['mileage'])

In [ ]:
## we will try to remove the mileage outliers by keeping the records upto 99% quartile

In [ ]:
mile_skewness=brand_mile['mileage'].skew(axis = 0, skipna = True)

In [ ]:
if mile_skewness>1:
    q = brand_mile['mileage'].quantile(0.99)
    brand_mile= brand_mile[brand_mile['mileage']<q]
elif mile_skewness<-1:
    q = brand_mile['mileage'].quantile(0.01)
    brand_mile= brand_mile[brand_data_mile['mileage']>q]

In [ ]:
sns.distplot(brand_mile['mileage'])

### Year Feature

In [ ]:
sns.distplot(brand_mile['year'])

In [ ]:
## we will try to remove the year outliers by keeping the records above 1% quartile

In [ ]:
brand_year=brand_mile.copy()
year_skewness=brand_year['year'].skew(axis = 0, skipna = True)

In [ ]:
if year_skewness>1:
    q = brand_year['year'].quantile(0.99)
    brand_year= brand_year[brand_year['year']<q]
elif year_skewness<-1:
    q = brand_year['year'].quantile(0.01)
    brand_year= brand_year[brand_year['year']>q]

In [ ]:
sns.distplot(brand_year['year'])

### Engine Size feature

In [ ]:
sns.distplot(brand_year['engineSize'])

In [ ]:
# We can drop Zero enginesizes or any above 3.5

In [ ]:
brand_engsZ=brand_year.copy()

In [ ]:
brand_engsZ = brand_engsZ[brand_engsZ.engineSize != 0]
brand_engsZ.drop(brand_engsZ.index[brand_engsZ['engineSize'] > 3.5], inplace = True)

In [ ]:
sns.distplot(brand_engsZ['engineSize'])

### Fuel Type

In [ ]:
brand_fuel=brand_engsZ.copy()
brand_fuel = brand_fuel[brand_fuel.fuelType != 'Hybrid']
brand_fuel = brand_fuel[brand_fuel.fuelType != 'Electric']
brand_fuel = brand_fuel[brand_fuel.fuelType != 'Other']

In [ ]:
vis_ftyp = brand_fuel.groupby(['fuelType']).count()['price'].sort_values(ascending=False)

In [ ]:
vis_ftyp.plot(kind='bar')
plt.show()

### Model

In [ ]:
brand_models=brand_fuel.copy()

In [ ]:
vis_models = brand_models.groupby(['model']).count()['price'].sort_values(ascending=False)

In [ ]:
vis_models.plot(kind='bar')
plt.show()

In [ ]:
# there are still some models with very few records
# I will filter those model with only 1 record for now

In [ ]:
brand_model_count=pd.DataFrame

brand_model_count=brand_model_count({'counts' : brand_models.groupby( [ "model"] ).size()}).reset_index()

min_count=1

counts_sum=brand_model_count['counts'].sum(axis=0)
brand_model_count.drop(brand_model_count[brand_model_count.counts >= min_count].index, inplace=True)

a=brand_model_count['model'].tolist()

for i in range (0,len(a)):
    brand_models.drop(brand_models[brand_models.model == a[i]].index,inplace=True)

vis_audi_models = brand_models.groupby(['model']).count()['price'].sort_values(ascending=False)

vis_audi_models.plot(kind='bar')
plt.show()

### Price

In [ ]:
sns.distplot(brand_models['price'])

In [ ]:
brand_models.boxplot(column=['price'],figsize=(15,5),patch_artist = True,notch ='True',vert=0)

In [ ]:
price_skewness=brand_models['price'].skew(axis = 0, skipna = True)
price_skewness

In [ ]:
# price is still highly skewed, we will try to visualize price with model and year for more better spotting of outliers

In [ ]:
brand_models.boxplot(column=['price'], by=['model','year'],figsize=(15,5),rot=90)

In [ ]:
# we need to remove price outliers based on model type and year (Assuming that those are the most effective features).
# we are going to calculate filter between 90% & 95% of each model_year price distribution 
# and exclude the rest in case the price variation is high (coefficient of varitaion >0.7)

In [ ]:
model_cstat=pd.DataFrame
model_mstat=pd.DataFrame
model_sdstat=pd.DataFrame
model_cstat=model_cstat({'cnt_price' : brand_models.groupby( [ "model","year"] ).count()["price"]}).reset_index()
model_mstat=model_mstat({'mean_price' : brand_models.groupby( [ "model","year"] ).mean()["price"]}).reset_index()
model_sdstat=model_sdstat({'std_price' : brand_models.groupby( [ "model","year"] ).std()["price"]}).reset_index()
model_stats1 = pd.merge(model_cstat, model_mstat, on=["model", "year"])
model_stats2 = pd.merge(model_stats1, model_sdstat, on=["model", "year"])

model_stats2['std_price'] = model_stats2['std_price'].fillna(0)

models_prices=brand_models.copy()

models_prices = models_prices.merge(model_stats2, how='inner', left_on=["model", "year"], right_on=["model","year"])
x=1.5
y=0.7
models_prices['var_coeff']=models_prices['std_price']/models_prices['mean_price']
models_prices['price_L95']=models_prices['mean_price']-x*models_prices['std_price']
models_prices['price_R95']=models_prices['mean_price']+x*models_prices['std_price']
models_prices['price_L95'] =models_prices['price_L95'].astype(int)
models_prices['price_R95'] =models_prices['price_R95'].astype(int)

def drop_price(models_prices):

    if (models_prices['var_coeff']>y) and (models_prices['cnt_price'] <4):
        return 'drop'
    elif (models_prices['price'] > models_prices['price_R95']) or (models_prices['price'] < models_prices['price_L95']):
        return 'drop'
    else:
        return 'keep'
models_prices['drop_price'] = models_prices.apply(drop_price, axis = 1)
models_prices = models_prices[models_prices.drop_price !='drop']

In [ ]:
models_prices.boxplot(column=['price'], by=['model','year'],figsize=(15,5),rot=90)

In [ ]:
price_skewness=models_prices['price'].skew(axis = 0, skipna = True)
price_skewness

In [ ]:
# Skewness slightly decreased and as you can see only the oultiers of specific model and year has been cleaned.

### Preocessing Checkpoint

In [ ]:
# Let us calculate the percentage of records removed out of the original dataset

In [ ]:
data_removed_perct=100*(len(brand_data)-len(models_prices))/len(brand_data)
data_removed_perct

In [ ]:
# Data removed is a bit high but accepted

In [ ]:
Brand_preprocessed=models_prices.copy()

In [ ]:
Brand_preprocessed=Brand_preprocessed.reset_index(drop=True)

In [ ]:
Brand_preprocessed.info()

## Re-arranging Columns

In [ ]:
Brand_preprocessed.columns.values

In [ ]:
cols=['price','model', 'year', 'transmission', 'mileage', 'fuelType',
      'engineSize', 'cnt_price', 'mean_price', 'std_price',
      'var_coeff', 'price_L95', 'price_R95', 'drop_price']

In [ ]:
brand_arranged=Brand_preprocessed.copy()
brand_arranged=Brand_preprocessed[cols]

In [ ]:
brand_arranged=brand_arranged.drop(['cnt_price', 'mean_price', 'std_price',
       'var_coeff', 'price_L95', 'price_R95', 'drop_price'], axis=1)

In [ ]:
brand_arranged.describe(include='all')

## Additional Graphs after Processing Data

### Year-Model Vs Mean Mileage

In [ ]:
year_model_mmile=pd.DataFrame
year_model_mmile=year_model_mmile({'mean_mileage' : brand_arranged.groupby( ["year",'model'] ).mean()["mileage"]}).reset_index().sort_values(by='year',ascending=True)

In [ ]:
model_set=sorted(set(year_model_mmile['model']))
plt.figure(figsize=(17,8))
for model in model_set:
     selected_data = year_model_mmile.loc[year_model_mmile['model'] == model]
     plt.plot(selected_data['year'], selected_data['mean_mileage'],marker = next(marker), label=model)
plt.xlabel('year')
plt.ylabel('mean mileage')
plt.legend()

In [ ]:
## Generally, mileage is higher for older cars

### Year-Model vs Mean Price

In [ ]:
year_model_mprice=pd.DataFrame
year_model_mprice=year_model_mprice({'mean_price' : brand_arranged.groupby( ["year",'model'] ).mean()["price"]}).reset_index().sort_values(by='year',ascending=True)

model_set=sorted(set(year_model_mprice['model']))
plt.figure(figsize=(17,8))
for model in model_set:
     selected_data = year_model_mprice.loc[year_model_mprice['model'] == model]
     plt.plot(selected_data['year'], selected_data['mean_price'],marker = next(marker), label=model)
plt.xlabel('year')
plt.ylabel('mean price')
plt.legend()
plt.show()

### Year-Model vs Min Price

In [ ]:
year_model_mnprice=pd.DataFrame
year_model_mnprice=year_model_mnprice({'min_price' : brand_arranged.groupby( ["year",'model'] ).min()["price"]}).reset_index().sort_values(by='year',ascending=True)

model_set=sorted(set(year_model_mnprice['model']))
plt.figure(figsize=(17,8))
for model in model_set:
     selected_data = year_model_mnprice.loc[year_model_mnprice['model'] == model]
     plt.plot(selected_data['year'], selected_data['min_price'],marker = next(marker), label=model)
plt.xlabel('year')
plt.ylabel('min price')
plt.legend()
plt.show()

### Year-Model vs Max Price

In [ ]:
year_model_mxprice=pd.DataFrame
year_model_mxprice=year_model_mxprice({'max_price' : brand_arranged.groupby( ["year",'model'] ).max()["price"]}).reset_index().sort_values(by='year',ascending=True)

model_set=sorted(set(year_model_mxprice['model']))
plt.figure(figsize=(17,8))
for model in model_set:
     selected_data = year_model_mxprice.loc[year_model_mxprice['model'] == model]
     plt.plot(selected_data['year'], selected_data['max_price'],marker = next(marker), label=model)
plt.xlabel('year')
plt.ylabel('max price')
plt.legend()
plt.show()

## Dropping Unwanted Fields

In [ ]:
brand_final=brand_arranged.drop(['year'], axis=1)

In [ ]:
# The remaining brands has been explored and preproceesed in the same manner and saved as csv

  # https://www.kaggle.com/enginsights/vw-preprocessing-and-exploration
  # https://www.kaggle.com/enginsights/vauxhall-preprocessing-and-exploration
  # https://www.kaggle.com/enginsights/bmw-preprocessing-and-exploration
  # https://www.kaggle.com/enginsights/audi-preprocessing-and-exploration
  # https://www.kaggle.com/enginsights/mercedes-preprocessing-and-exploration
  # https://www.kaggle.com/enginsights/skoda-preprocessing-and-exploration
  # https://www.kaggle.com/enginsights/toyota-preprocessing-and-exploration
  # https://www.kaggle.com/enginsights/hyundai-preprocessing-and-exploration

# All CSVs of processed data of all brands will be imported to separate notebook where a regression model is applied

# Please refer to 100K UK Used Cars Model

  # https://www.kaggle.com/enginsights/100k-uk-used-cars-model